# User-defined Exceptions

If the built-in exceptions are insufficient, create a user-defined exception.  Consider the following function which uses Heron's formula to compute the area of a triangle given the length of three sides:

In [ ]:
import math

def triangle_area(a, b, c):
    p = (a + b + c) / 2
    a = math.sqrt(p * (p - a) * (p - b) * (p - c))
    return a

In [ ]:
triangle_area(3, 4, 5)

If no such trinagle with these side lengths exists, a `ValueError` is raised from the attempt to find a real square root of a negative number:

In [ ]:
triangle_area(3, 4, 10)

Rather than the obscure math domain error message raised from the above, raise a more specific exception here which can carry more useful infromation in its payload.  A good start is to define a new exception class `TriangleError`

## A basic exception implementation

When creating a user-defined exception, subclass `Exception` rather than `BaseException`.  For a distinct exception type with basic facilities which can be raised and handled separately from other exception types, the most basic definition can suffice:

In [ ]:
class TriangleError(Exception):
    pass

The above is a fully functioning exception since it inherits complete implementations of `__init__()`, `__str__()` and `__repr__()`.  Modify the `triangle_area` function to identify illegal triangles:

In [ ]:
import math

class TriangleError(Exception):
    pass

def triangle_area(a, b, c):
    sides = sorted((a, b, c))
    if sides[2] > sides[0] + sides[1]:
        raise TriangleError("Illegal triangle")
    
    p = (a + b + c) / 2
    a = math.sqrt(p * (p - a) * (p - b) * (p -c))
    return a

In [ ]:
triangle_area(3, 4, 10)

## Enriching the payload

Modify the exception to accept more data about the putative triangle:

In [ ]:
import math

class TriangleError(Exception):

    def __init__(self, text, sides):
        super().__init__(text) # <- Message forwared to base class constructor
        self._sides = tuple(sides) # <- side lengths are stored in an instance attribute in the derived class

    @property
    def sides(self):
        return self._sides

    def __str__(self):
        return "'{}' for sides {}".format(self.args[0], self._sides)

    def __repr__(self):
        return "TriangleError({!r}, {!r}".format(self.args[0], self._sides)

The above exception now overrides `__init__()` and provides a construcotr which accepts a message and collecction of side lengths.  The message is forwarded to the base-class constructor for storage, and the side lengths are stored in an instance attribute in the derived class.

Side lengths are stored ina a tupe to prevent modification, and a read-only attribute is provided to access them.  `__str__()` and `__repr__()` methods are overriden using the `args` attribute from the base-class to retrieve the message string.

Remember to modify the constructor call for the exception:

In [ ]:
def triangle_area(a, b, c):
    sides = sorted((a, b, c))
    if sides[2] > sides[0] + sides[1]:
        raise TriangleError("Illegal triangle", sides)
    
    p = (a + b + c) / 2
    a = math.sqrt(p * (p - a) * (p - b) * (p -c))
    return a

Providing an illegal triangle into the function now produces a better error report:

In [ ]:
triangle_area(4, 10, 4)

With an appropriate handler in place, it is now possible to get access to the side lengths which caused the problem:

In [ ]:
try:
    triangle_area(3, 4, 10)
except TriangleError as e:
    print(e.sides)